In [18]:

pip install mysql-connector-python

     |████████████████████████████████| 307kB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 6.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#The below code takes any file from the online Kaggle repository "College Basketball Dataset" by Andrew Sundberg
# (which can be found here: https://www.kaggle.com/andrewsundberg/college-basketball-dataset), and writes it to a
# SQL database.


import getpass
from kaggle.api.kaggle_api_extended import KaggleApi #Kaggle API key needed for import
import zipfile
import kaggle
import pandas as pd
import mysql.connector


approved_files = ['cbb.csv', 'cbb13.csv', 'cbb14.csv', 'cbb15.csv', 'cbb16.csv', 'cbb17.csv', 'cbb18.csv', 'cbb19.csv', 'cbb20.csv', 'cbb21.csv']
#All Files in DataFrame


while True:
    new_file = input("Please input a valid file name from the database: ") #Prompt user for file name
    if new_file in approved_files:
        break
    else:
        print('Invalid file name. Valid file names consist of "cbb" + a year (13, 14, etc)')
        #Prompt user to input a different file name if the inputted one is not in the list

username = input("Please input your kaggle username credential: ")

api_key = getpass.getpass("Please input your custom kaggle API key: ")

sql_server = input("Please input the name of the SQL server you will be using: ")

sql_username = input("Please input your MySQL username credential: ")

sql_pwd = getpass.getpass("Please input your MySQL password (will be deleted from the system at the end): ")

sql_database = input("Please input the name of the SQL database you want to create: ")

api = KaggleApi()
api.authenticate() #import API

new_file = os.getenv('new_file')

api.dataset_download_file('andrewsundberg/college-basketball-dataset', file_name=new_file, path="./")


try:
    with zipfile.ZipFile(f'./{new_file}.zip', 'r') as zipref:
        zipref.extractall('./')
except: #unzip the folder kaggle gives you when downloading a file
    pass


data = pd.read_csv(f'./{new_file}')  #read in the file
df = pd.DataFrame(data)

df = df[['G', 'W', 'EFG_O', 'EFG_D']] #extract the 4 columns we want

df = df.sort_values('W', ascending=False) #sort by wins

df = df.head(100) #take first 100 values

print("This dataset consists of 4 columns (Games Played, Wins, Effective Field Goal Percentage (Offense) and Opponent Effective Field Goal Percentage) from the the 100 winningest Division 1 Programs from the given season. A sample can be found below:")

print(df.head(5)) #give user a look at the first 5 elements


df.to_csv('./df.csv')


def connection_func(sql_server, sql_username, sql_pwd, sql_database):
    new_conn = None
    try:
        new_conn = mysql.connector.connect( #function to create new mySQL connection, using user inputs
            host=sql_server,
            user=sql_username,
            passwd=sql_pwd,
            database=sql_database
        )
        print('Database connection successful')
    except mysql.connector.Error as err: #custom error message if user input is invalid
        print(f"One or more of the imputted values is invalid".format(err))

    return new_conn


def query_func(new_conn, new_query):
    new_cursor = new_conn.cursor()
    try:
        new_cursor.execute(new_query) #function to execute a query for the given connection
        new_conn.commit()
    except mysql.connector.Error as err: #custom error if query fails
        print(f"Something went wrong with the query:".format(err))


df = pd.read_csv('./df.csv') 


new_conn = connection_func(sql_server, sql_username, sql_pwd, sql_database) #run connection function on given information

extensionless_file = new_file.replace('.csv', '') #delete file extension

#create new table
new_query = f'CREATE TABLE {extensionless_file} (G float, W float, EFG_O float, EFG_D float)'
query_func(new_conn, new_query) #Run query on connection


create_cursor = new_con.cursor()
for row in list(range(0,len(df))):
    sql_df = f'INSERT INTO {f}(G, W, EFG_O, EFG_D) VALUES (%s,%s,%s,%s)'
    num_games = float(df['G'].iloc[row]) #insert the values we need into the sql object
    num_wins = float(df['W'].iloc[row])
    efg_offense = float(df['EFG_O'].iloc[row])
    efg_defense = float(df['EFG_D'].iloc[row])
    create_cursor.execute(sql_df, (num_games, num_wins, efg_offense, efg_defense))
new_conn.commit()


print("Your SQL database, " + str(sql_database) + " has been created!")
